# 🚀 SIAO-CNN-ORNN Nuclear Reactor Fault Detection Training

This notebook trains the **Self-Improved Aquila Optimizer enhanced CNN-ORNN** model for:
- IP-200 Nuclear Reactor Fault Detection
- Reliability Analysis

## 5-Stage Pipeline
```
Data → Windows → CNN + WKS → Concatenate → SIAO-ORNN → Softmax
```

## 1️⃣ Clone Repository & Install Dependencies

In [ ]:
# Clone the repository (update with your GitHub URL)
!git clone https://github.com/Sisoodiya/siao-cnn-ornn.git
%cd siao-cnn-ornn

# Install dependencies
!pip install -q torch numpy pandas scipy scikit-learn matplotlib openpyxl tqdm

## 2️⃣ Verify GPU & Environment

In [ ]:
import torch
import numpy as np
import random

# Set seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True

# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 3️⃣ Run Complete Training Pipeline

In [ ]:
# Import and run the training pipeline
from train_pipeline import run_complete_pipeline

# Train with default parameters
results = run_complete_pipeline(
    data_dir='data/',
    window_size=50,
    stride=25,
    cnn_embedding_dim=256,
    wks_dim=43,
    rnn_hidden_size=128,
    num_classes=6,
    test_size=0.2,
    wks_pop_size=15,      # Aquila optimizer for WKS
    wks_max_iter=30,
    siao_pop_size=20,     # SIAO for RNN weights
    siao_max_iter=50,
    bp_epochs=100,        # Backpropagation fine-tuning
    batch_size=32
)

print(f"\n{'='*60}")
print(f"Final Train Accuracy: {results['train_accuracy']:.4f}")
print(f"Final Val Accuracy:   {results['val_accuracy']:.4f}")

## 4️⃣ Reliability Analysis

In [ ]:
from stage5_inference.reliability import compute_reliability_report, plot_reliability_curves, plot_fault_distribution

# Compute reliability metrics from predictions
calculator, report = compute_reliability_report(
    results['val_preds'],
    time_step_hours=1.0,
    print_report=True
)

# Visualize
plot_reliability_curves(calculator)
plot_fault_distribution(results['val_preds'])

## 5️⃣ Save Trained Model

In [ ]:
import torch

# Save models
torch.save(results['cnn_model'].state_dict(), 'cnn_model.pth')
torch.save(results['ornn_trainer'].model.state_dict(), 'ornn_model.pth')

print("✅ Models saved:")
print("   - cnn_model.pth")
print("   - ornn_model.pth")

# Download from Colab
try:
    from google.colab import files
    files.download('cnn_model.pth')
    files.download('ornn_model.pth')
except:
    print("\nNot running in Colab - models saved locally.")

## 6️⃣ Classification Results Summary

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Class names
class_names = ['Steady State', 'Transient', 'PORV', 'SGTR', 'FWLB', 'RCP Failure']

# Classification report
print("Classification Report:")
print(classification_report(results['y_val'], results['val_preds'], target_names=class_names, digits=4))

# Confusion matrix
cm = confusion_matrix(results['y_val'], results['val_preds'])
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('SIAO-CNN-ORNN Confusion Matrix')
plt.tight_layout()
plt.show()